In [2]:
import subprocess
import json

# Check for MCP servers using common methods
def get_local_mcp_servers():
    servers = []
    
    # Method 1: Check if mcp command is available
    try:
        result = subprocess.run(['mcp', 'list'], capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            servers.extend(result.stdout.strip().split('\n'))
    except (subprocess.TimeoutExpired, FileNotFoundError):
        pass
    
    # Method 2: Check common MCP server locations
    import os
    common_paths = [
        '/usr/local/bin',
        '/opt/mcp/servers',
        os.path.expanduser('~/.mcp/servers'),
        os.path.expanduser('~/mcp-servers')
    ]
    
    for path in common_paths:
        if os.path.exists(path):
            try:
                for item in os.listdir(path):
                    if 'mcp' in item.lower() and os.path.isfile(os.path.join(path, item)):
                        servers.append(f"{path}/{item}")
            except PermissionError:
                continue
    
    # Method 3: Check running processes
    try:
        result = subprocess.run(['ps', 'aux'], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            for line in result.stdout.split('\n'):
                if 'mcp' in line.lower() and 'server' in line.lower():
                    servers.append(f"Running process: {line.strip()}")
    except (subprocess.TimeoutExpired, FileNotFoundError):
        pass
    
    return list(set(servers)) if servers else ["No MCP servers found locally"]

# Get and display MCP servers
local_mcp_servers = get_local_mcp_servers()
print("Local MCP Servers:")
for server in local_mcp_servers:
    print(f"- {server}")


Local MCP Servers:
- No MCP servers found locally


In [4]:
# # List all running Docker containers
# docker ps

# # Filter containers by name pattern (if MCP servers follow naming convention)
# docker ps --filter "name=mcp"

# # Show all containers with their ports
# docker ps --format "table {{.Names}}\t{{.Ports}}\t{{.Status}}"

# # Check specific container logs
# docker logs <container_name>

# # Inspect container details
# docker inspect <container_name>

# # List containers by image if you know the MCP server image
# docker ps --filter "ancestor=<mcp_image_name>"
# # a

In [8]:
import docker
import json

# Connect to Docker client
client = docker.from_env()

# Find the mcp_neo4j_data_modeling container
container_name = "mcp_neo4j_data_modeling"

try:
    container = client.containers.get(container_name)
    
    # Get container information
    container_info = container.attrs
    
    # Extract services information
    if 'Config' in container_info and 'ExposedPorts' in container_info['Config']:
        exposed_ports = container_info['Config']['ExposedPorts']
        print(f"Exposed ports for {container_name}:")
        for port in exposed_ports:
            print(f"  - {port}")
    
    # Get network settings
    if 'NetworkSettings' in container_info:
        networks = container_info['NetworkSettings']['Networks']
        print(f"\nNetworks for {container_name}:")
        for network_name, network_info in networks.items():
            print(f"  - Network: {network_name}")
            if 'IPAddress' in network_info:
                print(f"    IP: {network_info['IPAddress']}")
    
    # Get port bindings
    if 'NetworkSettings' in container_info and 'Ports' in container_info['NetworkSettings']:
        ports = container_info['NetworkSettings']['Ports']
        print(f"\nPort bindings for {container_name}:")
        for internal_port, external_bindings in ports.items():
            if external_bindings:
                for binding in external_bindings:
                    print(f"  - {internal_port} -> {binding['HostIp']}:{binding['HostPort']}")
            else:
                print(f"  - {internal_port} (not bound to host)")
    
    # Get environment variables that might indicate services
    if 'Config' in container_info and 'Env' in container_info['Config']:
        env_vars = container_info['Config']['Env']
        print(f"\nEnvironment variables for {container_name}:")
        for env in env_vars:
            if any(keyword in env.upper() for keyword in ['PORT', 'SERVICE', 'HOST', 'URL']):
                print(f"  - {env}")

except docker.errors.NotFound:
    print(f"Container '{container_name}' not found")
except Exception as e:
    print(f"Error: {e}")



Networks for mcp_neo4j_data_modeling:
  - Network: config_neo4j-network
    IP: 172.20.0.6

Port bindings for mcp_neo4j_data_modeling:

Environment variables for mcp_neo4j_data_modeling:
  - NEO4J_URL=bolt://neo4j:7687


In [7]:

import subprocess
import sys

# Install docker
subprocess.check_call([sys.executable, "-m", "pip", "install", "docker"])

# json and subprocess are built-in modules, no installation needed
import docker
import json
import subprocess


In [11]:
!pip install mcp_neo4j_data_modeling


  Using cached pyperclip-1.9.0.tar.gz (20 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 7.0 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.7/630.7 kB 8.0 MB/s  0:00:00
  DEPRECATION: Building 'pyperclip' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pyperclip'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11103 sha256=98d361419183359f61ddbf9275a6d0daee8314aa5c201705eabb8b4c6a0d1086
  Stored in directory: /Users/ravikiranponduri/Library/Caches/pip/wheels/e8/e7/56/591cb88ba1783b38c40d584026e766aac9c3a048e34128ce8b


In [9]:
import os
os.environ['NEO4J_URL'] = 'bolt://neo4j:7687'


In [12]:
from mcp_neo4j_data_modeling import Neo4jDataModeler

# Initialize the data modeler
modeler = Neo4jDataModeler()

# Define entities and their relationships
entities = {
    "Order": {
        "properties": ["order_id"],
        "constraints": ["order_id:UNIQUE"]
    },
    "OrderLine": {
        "properties": ["line_number", "quantity"],
        "constraints": ["line_number:INTEGER", "quantity:INTEGER"]
    },
    "Item": {
        "properties": ["item"],
        "constraints": ["item:UNIQUE"]
    }
}

relationships = [
    ("Order", "HAS_LINE", "OrderLine"),
    ("OrderLine", "CONTAINS", "Item")
]

# Generate schema
schema = modeler.generate_schema(entities, relationships)
print("Recommended Neo4j Schema:")
print(schema)


ImportError: cannot import name 'Neo4jDataModeler' from 'mcp_neo4j_data_modeling' (/opt/anaconda3/envs/mcp-neo4j-env/lib/python3.11/site-packages/mcp_neo4j_data_modeling/__init__.py)